<a href="https://colab.research.google.com/github/aryanmikaeili/cmpt732_material/blob/master/python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPT 732 - Fall 2022
# Soving systems of linear equations

__content creator:__ Aryan Mikaeili

In [3]:
#@title Import modules

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/course-content/master/nma.mplstyle")

import requests
import shutil

import cv2
from scipy.sparse import csr_matrix

## Practice 1

Solve the linear equation in the slides. To complete this practice you should:

1. form matrix $A$
2. form vector $b$
3. solve the equation using numpy

In [2]:
##form matrix A

A = ...

##form vector b

b = ...

##solve the linear equation

x = ...
print(x)

Ellipsis


In [ ]:
#@title Solution

##form matrix A
A = np.array([[0, 0, 0, 1, 1, 0], [0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1], [1, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 0], [0, 0, 1, -1, 0, 0]])

##form vector b
b = np.array([3, 5, 7, 9, 11, 5])

##solve the linear equation

x = np.linalg.pinv(A) @ b
print('answer is: ', x)
print('error is: {:.4f}'.format(np.linalg.norm(A@x - b)))


## Practice 2

Solve the linear equation in the slides. To complete this practice you should:

1. form matrix $A$
2. form vector $b$
3. solve the equation using numpy

In [ ]:
##form matrix A

A = ...

##form vector b

b = ...

##solve the linear equation

x = ...
print(x)

In [ ]:
#@title Solution

##form matrix A
A = np.array([[1, 1], [3, -2], [4, -1]])

##form vector b
b = np.array([2, 1, 7])

##solve the linear equation

x = np.linalg.pinv(A) @ b
print('answer is: ', x)
print('error is: {:.4f}'.format(np.linalg.norm(A@x - b)))


In [ ]:
#@title Reconstructing curves from gradients

#define curve
x = np.arange(0, 100)
y = np.sin(x * x / 500)

#plot curve
plt.plot(x, y)
plt.show()

In [ ]:
#@title define gradients
#define gradients:

dy = y[1:] - y[:-1]
print(dy.shape)

#plot gradients
plt.plot(x[:-1], dy)
plt.show()

In [ ]:
#@title define and solve system of linear equations

#define A
A = np.zeros((100, 100))
for i in range(99):
  A[i, i:i+2] = [-1, 1]

#Add one constraint
A[-1, 0] = 1

#b is dy + one constraint
b = np.concatenate([dy, [0]])

#solve equation

y_hat = np.linalg.pinv(A) @ b


#print error
print('error = {:.4f}'.format(np.linalg.norm(y_hat - y)))
#plot y_hat
plt.plot(x, y_hat)
plt.show()

## Image reconstruction from gradient domain

Similar to what we did with the 1-dimensional curve, we can also reconstruct an image from  its gradient domain. Assume that you have an $H\times W$ image that you want to reconstruct and what you have are its gradients in the $x$ and $y$ directions. We know that:

$$g_x(i, j) = p(i, j + 1) - p(i, j)$$
$$g_y(i, j) = p(i + 1, j) - p(i, j)$$

where $g_x(i, j)$ and $g_y(i, j)$ are gradients of the image in the $x$ and $y$ direction in pixel $(i, j)$. You can additionally add a contorl point to adjust brightness.

Naturally, whith the equations above, you can form a system of linear equations. In this system we will have $2(H - 1)(W - 1) + 1$ equations and $HW$ variables(Why?). Matrix $A$ will have shape $2(H - 1)(W - 1) + 1 \times HW$ and $x$ and $b$ will be $HW \times 1$ vectors.


In [ ]:
#@title Run cell to download
response = requests.get("https://e2.365dm.com/08/10/800x600/PaoloMaldini_1369005.jpg?20081023161130", stream=True)
with open('maldini.jpg', 'wb') as f:
  shutil.copyfileobj(response.raw, f)

image = Image.open('maldini.jpg').convert('L')
image = image.crop((150, 100, 400 ,400))
image = image.resize((50, 50))
image = np.array(image).astype('int')

plt.axis('off')
plt.imshow(image, cmap = 'gray')
plt.show()

In [ ]:
##form matrix A and vector b
H, W = image.shape

A = np.zeros((2 * (H - 1) * (W - 1), H * W))
b = np.zeros(2 * (H - 1) * (W - 1))

##write your code here
...

##Add constraint that controls brightness
...

##solve system of equations
image_hat = ...

##plot image
image_hat = image_hat.reshape(50, 50)
plt.axis('off')
plt.imshow(image_hat, cmap = 'gray')
plt.show()

In [ ]:
#@title Solution

##form matrix A and vector b
H, W = image.shape

A = np.zeros((2 * (H - 1) * (W - 1), H * W))
b = np.zeros(2 * (H - 1) * (W - 1))


counter = 0
for i in range(H - 1):
  for j in range(W - 1):
    A[counter,  i * W + j] = -1
    A[counter,  i * W + j + 1] = 1
    A[counter + 1, i * W + j] = -1
    A[counter + 1, (i + 1) * W + j] = 1
    b[counter] = image[i, j + 1] - image[i, j]
    b[counter + 1] = image[i + 1, j] - image[i, j]

    counter += 2


##Add constraint that controls brightness
const = np.zeros(H * W)
const[0] = 1

A = np.row_stack([A, const])
b = np.concatenate([b, [25]])
    
##solve system of equations
image_hat = np.linalg.pinv(A) @ b

##plot image
image_hat = image_hat.reshape(50, 50)
plt.axis('off')
plt.imshow(image_hat, cmap = 'gray')
plt.show()